In [38]:
import os
from dotenv import load_dotenv
import requests
import json

# Load environment variables from .env file
load_dotenv()

In [108]:
class NotionCrud:
    def __init__(
        self,
        notion_token = None,
        page_id = None,
        database_id = None
    ):
        self.url = "https://api.notion.com/v1"

        if notion_token:
            self.NOTION_API_KEY = notion_token
        else:    
            self.NOTION_API_KEY = os.getenv('NOTION_SECRET')
        
        self.headers = {
            "Authorization": f"Bearer {self.NOTION_API_KEY}",
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
        }

        if page_id:
            self.PAGE_ID = page_id
        else:
            self.PAGE_ID = os.getenv('NOTION_DATABASE')

        if database_id:
            self.DATABASE_ID = database_id
        else:
            self.DATABASE_ID = self.get_inline_database_id()
        
    def get_inline_database_id(self):
        """
        Fetches the database ID of an inline database on a Notion page.

        Parameters:
        - page_id (str): The ID of the Notion page containing the inline database.
        - notion_token (str): Your Notion API integration token.

        Returns:
        - str: The database ID of the inline database, or None if not found.
        """
        url = f"https://api.notion.com/v1/blocks/{self.PAGE_ID}/children"

        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            data = response.json()

            # Search for a block of type 'child_database'
            for block in data.get("results", []):
                if block["type"] == "child_database":
                    return block["id"]

            print("No inline database found on the page. Creating new database.")
            database_json = self.create_database("Talk2Doc Job Applications", True)
            return database_json['id']

        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

    def create_database(
        self,
        PAGE_TITLE,
        IS_INLINE
    ):
        url = f"{self.url}/databases"
        
        payload = {
            "parent": {"type": "page_id", "page_id": self.PAGE_ID},
            "title": [{"type": "text", "text": {"content": PAGE_TITLE}}],
            "is_inline": IS_INLINE,
            "properties": {
                "Company": {"title": {}},
                "Position": {"rich_text": {}},
                "Data of Application": {"date": {}},
                "Status": {
                    "select": {
                        "options": [
                            {"name": "Applied", "color": "blue"},
                            {"name": "OA", "color": "yellow"}, 
                            {"name": "Rejected", "color": "red"},
                            {"name": "Interview", "color": "green"},
                            {"name": "Offer", "color": "purple"}
                        ]
                    }
                },
                "Deadline": {"date": {}},
                "Notes": {"rich_text": {}}
            }
        }

        response = requests.post(url, headers=self.headers, json=payload)

        if response.status_code == 200:
            print("Database created successfully!")
            # print(json.dumps(response.json(), indent=2))
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            print(response.text)
            return None

    def create_database_entry(
        self,
        title=None,
        position=None,
        notes=None,
        status=None,
        deadline=None,
        data_of_application=None
    ):
        """
        Dynamically creates an entry in a Notion database.

        Parameters:
        - database_id (str): The ID of the Notion database.
        - notion_token (str): Your Notion API integration token.
        - title (str): The value for the 'Company' (title) property.
        - position (str): The value for the 'Position' (rich_text) property.
        - notes (str): The value for the 'Notes' (rich_text) property.
        - status (str): The value for the 'Status' (select) property.
        - deadline (str): The value for the 'Deadline' (date) property in ISO format (YYYY-MM-DD).
        - data_of_application (str): The value for the 'Data of Application' (date) property in ISO format (YYYY-MM-DD).

        Returns:
        - dict: The created page response from Notion.
        """
        url = f"{self.url}/pages"

        # Build the properties dynamically
        properties = self.get_properties_dict(title, position, notes, status, deadline, data_of_application)

        # Build the payload
        payload = {
            "parent": {"database_id": self.DATABASE_ID},
            "properties": properties
        }

        try:
            response = requests.post(url, headers=self.headers, json=payload)
            response.raise_for_status()
            print("Page created successfully!")
            # return response.json()
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

    def get_payload(
        self,
        title,
        position,
        status,
        deadline,
        data_of_application,
        notes
    ):
        filters = []
        if title:
            filters.append({
                "property": "Company",
                "title": {
                    "equals": title
                }
            })
        if position:
            filters.append({
                "property": "Position",
                "rich_text": {
                    "contains": position
                }
            })
        if status:
            filters.append({
                "property": "Status",
                "select": {
                    "equals": status
                }
            })
        if deadline:
            filters.append({
                "property": "Deadline",
                "date": {
                    "equals": deadline
                }
            })
        if data_of_application:
            filters.append({
                "property": "Data of Application",
                "date": {
                    "equals": data_of_application
                }
            })
        if notes:
            filters.append({
                "property": "Notes",
                "rich_text": {
                    "contains": notes
                }
            })

        # If no filters are provided, query all pages
        payload = {"filter": {"and": filters}} if filters else {}
        return payload

    def retrieve_page_dynamic(
        self,
        title=None,
        position=None,
        status=None,
        deadline=None,
        data_of_application=None,
        notes=None
    ):
        """
        Dynamically retrieves a page from a Notion database based on provided properties.

        Parameters:
        - database_id (str): The ID of the Notion database.
        - notion_token (str): Your Notion API integration token.
        - title (str): Filter by 'Company' (title) property.
        - position (str): Filter by 'Position' (rich_text) property.
        - status (str): Filter by 'Status' (select) property.
        - deadline (str): Filter by 'Deadline' (date) property in ISO format (YYYY-MM-DD).
        - data_of_application (str): Filter by 'Data of Application' (date) property in ISO format (YYYY-MM-DD).
        - notes (str): Filter by 'Notes' (rich_text) property.

        Returns:
        - list: The results of the query matching the filters.
        """
        url = f"{self.url}/databases/{self.DATABASE_ID}/query"

        # Build the filter dynamically
        payload = self.get_payload(title, position, status, deadline, data_of_application, notes)

        try:
            response = requests.post(url, headers=self.headers, json=payload)
            response.raise_for_status()
            return response.json().get("results", [])
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return []

    def get_properties_dict(
        self,
        title,
        position,
        notes,
        status,
        deadline,
        data_of_application
    ):
        properties = {}
        if title:
            properties["Company"] = {
                "title": [
                    {
                        "type": "text",
                        "text": {"content": title},
                    }
                ]
            }
        if position:
            properties["Position"] = {
                "rich_text": [
                    {
                        "type": "text",
                        "text": {"content": position},
                    }
                ]
            }
        if notes:
            properties["Notes"] = {
                "rich_text": [
                    {
                        "type": "text",
                        "text": {"content": notes},
                    }
                ]
            }
        if status:
            properties["Status"] = {
                "select": {"name": status}
            }
        if deadline:
            properties["Deadline"] = {
                "date": {"start": deadline, "end": None}
            }
        if data_of_application:
            properties["Data of Application"] = {
                "date": {"start": data_of_application, "end": None}
            }
        return properties

    def get_page_id_to_update(self, title, position, status, deadline, data_of_application, notes):
        pages = self.retrieve_page_dynamic(title=title, position=position, status=None, deadline=None, data_of_application=None, notes=None)
        return pages[0]['id']

    def update_notion_page(
        self,
        page_id,
        title=None,
        position=None,
        notes=None,
        status=None,
        deadline=None,
        data_of_application=None
    ):
        """
        Dynamically updates properties of a Notion page based on the provided arguments.

        Parameters:
        - page_id (str): The ID of the Notion page to update.
        - notion_token (str): Your Notion API integration token.
        - title (str): The value to update the 'Company' (title) property.
        - position (str): The value to update the 'Position' (rich_text) property.
        - notes (str): The value to update the 'Notes' property.
        - status (str): The value to update the 'Status' (select) property.
        - deadline (str): The value to update the 'Deadline' (date) property in ISO format (YYYY-MM-DD).
        - data_of_application (str): The value to update the 'Data of Application' (date) property in ISO format (YYYY-MM-DD).

        Returns:
        - dict: The updated page response from Notion.
        """

        url = f"{self.url}/pages/{page_id}"

        # Build the properties dynamically
        properties = self.get_properties_dict(title, position, notes, status, deadline, data_of_application)

        # Send the PATCH request if there are updates to make
        if properties:
            payload = {"properties": properties}
            try:
                response = requests.patch(url, headers=self.headers, json=payload)
                response.raise_for_status()
                return response.json()
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
                return None
        else:
            print("No properties to update.")
            return None


In [109]:
notion_user = NotionCrud(page_id=PAGE_ID)

In [98]:
notion_user.create_database_entry(title="Apple", position="Software Engineer", status="Applied", deadline=None, data_of_application="2024-11-18", notes="Apple application notes")

Page created successfully!


In [ ]:
page_id = notion_user.get_page_id_to_update(title="Apple", position="Software Engineer", status=None, deadline=None, data_of_application=None, notes=None)
notion_user.update_notion_page(page_id=page_id, title="Apple", position="Software Engineer", status="OA", deadline="2024-11-25", data_of_application="2024-11-18", notes="Apple application updated notes")